### Inputs

In [1]:
# PAPERMILL
wse_file = 's3://pfra/RiskAssessment/DC/Results/F02/WSE_DC_F02_Uniform.csv'
breach_prob_file = 's3://pfra/RiskAssessment/DC/BreachAnalysis/DC_F02_raw_prob_table.csv'
weights_file = 's3://pfra/RiskAssessment/DC/BreachAnalysis/DC_F02.xlsx'

In [2]:
import sys; sys.path.append('../core')
from risk_refactor import *
from importlib import reload
print('Working Directory = {}'.format(os.getcwd()))

Working Directory = C:\Users\slawler\GitRepos\probmod-tools\core


### Adjust curves for current FEMA implementation

In [3]:
defaultHazusDDFn_path = '../risk/hazusdepthdmgfns/Building_DDF_Full_LUT_Hazus3p0.json'
df_BDDFn = pd.read_json(defaultHazusDDFn_path, orient = 'index')
df_BDDFn = hazusID_to_depth(df_BDDFn)
df_agg = aggregate_ddf_curves(df_BDDFn, curve_groups, plot=False)

# set curve for single family 1 story no basement
df_agg.loc[-1, 'singFam_1Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_1Story_NoBasement']= 0.6

# set curve for single family 2 story no basement
df_agg.loc[-1, 'singFam_2Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_2Story_NoBasement']= 0.6

# set curve for single family 3 story no basement
df_agg.loc[-1, 'singFam_3Story_NoBasement']= 0.0
df_agg.loc[-0.000000000001, 'singFam_3Story_NoBasement']= 0.0

# set curve for mobile home
df_agg.loc[-0.000000000001, 'mobileHome']= 0.75

df_agg = df_agg.sort_index()

In [4]:
test_locations = ['87C5W2MX+5G9GF53',
                    '87C5W2MX+6G69853',
                    '87C5W2MX+6M47X34',
                    '87C5W2MX+9JWGM55',
                    '87C5W2MX+FRP9M22',
                    '87C5W2MX+CP6Q933',
                    '87C5W2RV+FQMRF34',
                    '87C5W2RW+F7G5222',
                    '87C5W2RW+FFH8R33',
                    '87C5W2RV+FWGGP35',
                    '87C5W2RV+GHC4345',
                    '87C5W2RW+G5F9R24',
                    '87C5W2RW+GG4R953',
                    '87C5W2RV+GQX3R45',
                    '87C5W2RV+GW9X234',
                    '87C5W2RV+HRJM344',
                    '87C5W2RW+GHRP625',
                    '87C5W2RW+H3PVR53',
                    '87C5W2RV+R9G7534',
                    '87C5W2RW+H43P443',
                    '87C5W2RW+H85V532',
                    '87C5W2RV+MRMJV34',
                    '87C5W2RV+MVMXW25',
                    '87C5W2RV+RGR9R32',
                    '87C5W2RV+MX39F45',
                    '87C5W2RW+H9V9525',
                    '87C5W2RV+RW47454',
                    '87C5W2RW+HHHQ855',
                    '87C5W2RV+PG7VG42',
                    '87C5W2RV+PHF5Q45',
                    '87C5W2RV+VXFH525',
                    '87C5W2RV+PJP2M52',
                    '87C5W2RW+J29M725',
                    '87C5W2RV+WPJ5252',
                    '87C5W2RW+J6VH724',
                    '87C5W2RW+JCJCH23',
                    '87C5W2RV+PMM9C24',
                    '87C5W2RV+PPX3C25']

In [5]:
dfwse = pd.read_csv(wse_file, index_col='plus_code')
dfw = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
dfbp = pd.read_table(breach_prob_file, index_col=0)
for col in dfbp.columns:
    dfbp.rename(columns={col:col.split('_')[1]}, inplace=True)

In [77]:
for event in e.event_weights.index:
    #print(event)
    event_map['NBR_{}'.format(event)] = e.event_weights.loc[event, 'Overall Weight']

In [84]:
for i, pcode in enumerate(test_locations[0:30]):
    # Initialize loss a point to zero
    aal = 0
    #dfwse = the row of results from dfwse
    event_data = dfwse.loc[pcode]
    
    p = FluvialPoint(pcode, event_data)
    
    e = FluvialEvents(dfw, dfbp)
    
    breaches  = p.breach_influence
    weights_map[p.plus_code] = {}
    
    if len(breaches) >0:
        breach_prob = {}
        for b in e.breach_locations:
            breach_prob[b]= e.breach_probs(b).to_dict()

        event_map = {}
        
        for idx, b in enumerate(breaches):
            for event in e.event_weights.index:
                event_map['{}'.format(b)] = breach_prob[b][event]
                test_breach_probs = calculate_breach_weights(event, event_map)

                test_breach_weights = {}
                for br_ev, prob in test_breach_probs.items():
                    ev = br_ev.split('_')[1]
                    wt = prob * e.event_weights.loc[ev, 'Overall Weight']
                    test_breach_weights[br_ev] = wt 

                for k, v in test_breach_weights.items():
                    weights_map[p.plus_code][k] = v
    else:
        for event in e.event_weights.index:
            weights_map[p.plus_code]['NBR_{}'.format(event)] = e.event_weights.loc[event, 'Overall Weight']
                
    bld_lmt = p.attributes['limitbuilding']
    bld_ded = p.attributes['deductiblebuilding']
    dmg_code = p.attributes['damage_code']
    ffh = p.attributes['firstfloorheight']

    loss_func = interp1d(df_agg.index, df_agg[dmg_code])

    depth_in_building = p.raw_wses - (p.elevation + p.attributes.firstfloorheight)

    relevent_events = [e for b in p.breach_influence for e in p.events if b in e]

    # Add nbr events
    for event in p.nbr_events:
        relevent_events.append(event)

    for event in relevent_events:
        event_depth_in_building = depth_in_building[event]

        if event_depth_in_building > df_agg.index.min():

            if event_depth_in_building > df_agg.index.max():
                event_depth_in_building = highest_ddf_elev

            percent_loss = loss_func(event_depth_in_building)/100
            loss_val = bld_lmt * percent_loss - bld_ded

            if loss_val > 0:
                try:
                    aal += loss_val * weights_map[p.plus_code][event]
                except KeyError as error:
                    print('Key Errror', error)
                    break

    print(i, pcode, aal)

0 87C5W2MX+5G9GF53 0.8010403554634713
1 87C5W2MX+6G69853 1.7931507462507792
2 87C5W2MX+6M47X34 37.311378596143896
3 87C5W2MX+9JWGM55 14.801945817670743
4 87C5W2MX+FRP9M22 0.7856212827536633
5 87C5W2MX+CP6Q933 29.82768191685065
6 87C5W2RV+FQMRF34 2.1910534185967787
7 87C5W2RW+F7G5222 8.037105510328502
8 87C5W2RW+FFH8R33 7.909668015372577
9 87C5W2RV+FWGGP35 2.9594282715034232
10 87C5W2RV+GHC4345 0
11 87C5W2RW+G5F9R24 7.6546307566310725
12 87C5W2RW+GG4R953 12.982804290771362
13 87C5W2RV+GQX3R45 1.5281650756552807
14 87C5W2RV+GW9X234 2.881820563323201
15 87C5W2RV+HRJM344 0.7638505067742825
16 87C5W2RW+GHRP625 14.317262574914439
17 87C5W2RW+H3PVR53 4.218165038825275
18 87C5W2RV+R9G7534 0
19 87C5W2RW+H43P443 6.4589193604528905
20 87C5W2RW+H85V532 9.853341932027753
21 87C5W2RV+MRMJV34 2.2872961965336076
22 87C5W2RV+MVMXW25 2.8083129825410835
23 87C5W2RV+RGR9R32 1.3542232166031103
24 87C5W2RV+MX39F45 4.822444015648348
25 87C5W2RW+H9V9525 9.716112225521101
26 87C5W2RV+RW47454 3.0459214427302395